# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas 
import pandas as pd
import requests
import geoviews as gv
import geoviews.operation as gvo

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df['Date'] = pd.to_datetime(city_data_df['Date'], unit='s').dt.strftime('%Y-%m-%d')

# Display sample data
city_data_df.head()
city_data_df.count()

City_ID       569
City          569
Lat           569
Lng           569
Max Temp      569
Humidity      569
Cloudiness    569
Wind Speed    569
Country       566
Date          569
dtype: int64

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    color = 'City',
    geo = True,
    tiles = "OSM",
    legend='right',
    frame_width = 900,
    frame_height = 600,
    size='Humidity',
    scale= 0.5
 )

# Display the map
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df[
    (city_data_df['Max Temp'] > 17) &
    (city_data_df['Max Temp'] < 22) &
    (city_data_df['Wind Speed'] < 6) &
    (city_data_df['Cloudiness'] > 7)]
criteria_df = criteria_df.dropna()

# Display sample data
criteria_df
criteria_df.count()



City_ID       45
City          45
Lat           45
Lng           45
Max Temp      45
Humidity      45
Cloudiness    45
Wind Speed    45
Country       45
Date          45
dtype: int64

### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = criteria_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
2,waitangi,NZ,-43.9535,-176.5597,93,
16,fort bragg,US,35.1390,-79.0060,88,
59,malanje,AO,-9.5402,16.3410,94,
62,urangan,AU,-25.3000,152.9000,84,
94,jamestown,US,42.0970,-79.2353,97,
95,luwingu,ZM,-10.2621,29.9271,99,
105,santa maria,BR,-29.6842,-53.8069,88,
140,senanga,ZM,-16.1167,23.2667,81,
142,tsiombe,MG,-25.3000,45.4833,87,
166,gonabad,IR,34.3529,58.6837,34,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accomodation.hotel',
    'apiKey': geoapify_key,
    'limit': 20  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
waitangi - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
malanje - nearest hotel: No hotel found
urangan - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
luwingu - nearest hotel: No hotel found
santa maria - nearest hotel: No hotel found
senanga - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
gonabad - nearest hotel: No hotel found
longmen - nearest hotel: No hotel found
lata - nearest hotel: No hotel found
nakuru - nearest hotel: No hotel found
montepuez - nearest hotel: No hotel found
tolanaro - nearest hotel: No hotel found
vila franca do campo - nearest hotel: No hotel found
thabazimbi - nearest hotel: No hotel found
sanctuary point - nearest hotel: No hotel found
malpica - nearest hotel: No hotel found
naze - nearest hotel: No hotel found
corinto - nearest hotel: No hotel found
concordia - nearest hotel: No hotel found
gowurdak - nearest hotel: No hotel found
itoman - nearest hotel: N

,City,Country,Lat,Lng,Humidity,Hotel Name
2,waitangi,NZ,-43.9535,-176.5597,93,No hotel found
16,fort bragg,US,35.1390,-79.0060,88,No hotel found
59,malanje,AO,-9.5402,16.3410,94,No hotel found
62,urangan,AU,-25.3000,152.9000,84,No hotel found
94,jamestown,US,42.0970,-79.2353,97,No hotel found
95,luwingu,ZM,-10.2621,29.9271,99,No hotel found
105,santa maria,BR,-29.6842,-53.8069,88,No hotel found
140,senanga,ZM,-16.1167,23.2667,81,No hotel found
142,tsiombe,MG,-25.3000,45.4833,87,No hotel found
166,gonabad,IR,34.3529,58.6837,34,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_names = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=900,
    frame_height=600,
    color= 'City',
    size= 'Humidity',
    hover_cols = ["Hotel Name", "Country"]
    
# Display the map
hotel_names

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)